In [4]:
import pandas as pd
import json

In [5]:
df_communes = pd.read_json('https://geo.api.gouv.fr/communes')
df_epci = pd.read_json('https://geo.api.gouv.fr/epcis')
df_departements = pd.read_json('https://geo.api.gouv.fr/departements')
df_regions = pd.read_json('https://geo.api.gouv.fr/regions')

In [6]:
df_communes

,nom,code,codeDepartement,siren,codeEpci,codeRegion,codesPostaux,population
0,L'Abergement-Clémenciat,01001,01,210100012.0,200069193.0,84,[01400],859.0
1,L'Abergement-de-Varey,01002,01,210100020.0,240100883.0,84,[01640],273.0
2,Ambérieu-en-Bugey,01004,01,210100046.0,240100883.0,84,[01500],15554.0
3,Ambérieux-en-Dombes,01005,01,210100053.0,200042497.0,84,[01330],1917.0
4,Ambléon,01006,01,210100061.0,200040350.0,84,[01300],114.0
...,...,...,...,...,...,...,...,...
35024,Touho,98830,988,NaN,NaN,988,[98831],2380.0
35025,Voh,98831,988,NaN,NaN,988,[98833],2856.0
35026,Yaté,98832,988,NaN,NaN,988,[98834],1667.0
35027,Kouaoua,98833,988,NaN,NaN,988,[98818],1304.0


In [24]:
df_epci["code"] = df_epci["code"].astype(str)

In [27]:
df_communes.codeEpci = df_communes.codeEpci.fillna(0).astype(int).astype(str)
df = df_communes.merge(df_epci, left_on='codeEpci', right_on='code')

In [36]:
df_prod = pd.read_csv("/Users/raphaelcourivaud/dev/zlv/dumps/bbuyufrdks9xrduy8zbupublic_establishments.csv")

In [37]:
df_prod

,id,siren,name,available,localities_geo_code,kind,updated_at,source
0,40a183f3-13da-44c4-b986-d691a96e2376,200067874,CA de Haguenau,True,"{67087,67497,67328,67023,67033,67035,67039,670...",CA,2024-05-29 06:54:06.188+00,seed
1,e7f6793e-b92b-4d44-9703-991d99ed8fe5,217801729,Commune de Conflans-Sainte-Honorine,True,{78172},Commune,2024-05-31 11:50:03.245+00,seed
2,a5ee625a-6a9a-4563-b5f6-d66f7d2ed58c,200036549,CC Val de Gray,True,"{70018,70394,70024,70026,70030,70032,70041,700...",CC,2024-06-04 07:16:55.308+00,seed
3,1afedf8e-b415-4eca-94c1-137268f7600c,200069052,CA Grand Belfort,True,"{90007,90103,90104,90001,90002,90004,90008,900...",CA,2023-04-20 13:27:52.527296+00,seed
4,1cfcb28a-703d-45f6-b261-956bda963f7b,215600834,Commune de Hennebont,True,{56083},Commune,2024-06-04 13:25:21.089+00,seed
...,...,...,...,...,...,...,...,...
36662,d15a2709-08f3-426d-9504-8e38dc2fff50,218500650,Commune de Chavagnes-en-Paillers,True,{85065},Commune,2023-04-20 13:27:52.527296+00,seed
36663,d66be947-fcfe-4c0f-bf5b-9b372e7f9115,242900751,CC du Pays de Landivisiau,True,"{29301,29010,29038,29068,29074,29097,29105,291...",CC,2023-04-20 13:27:52.527296+00,seed
36664,25c89afe-16b9-4220-a403-91da62c5f61d,200072882,CC de Vie et Boulogne,True,"{85130,85003,85006,85015,85019,85055,85086,850...",CC,2023-04-20 13:27:52.527296+00,seed
36665,191e0d78-c5cf-4620-99b8-84b99bee4bc0,212200810,Commune de Hillion,True,{22081},Commune,2023-04-20 13:27:52.527296+00,seed


In [28]:
cols_to_unpack = ["epci", "departement", "region"]
name_in_json_object = ["code", "nom"]

In [34]:
df = df_communes

In [50]:
import pandas as pd

def string_to_set(s):
    if pd.isna(s) or s == '':
        return set()
    return set(s.replace('{', '').replace('}', '').split(','))

# Préparation données
# df_prod['localities_geo_code'] = df_prod['localities_geo_code'].apply(string_to_set)

# Mapping SIREN -> codes INSEE
siren_to_insee = {
    row['siren']: row['localities_geo_code'] 
    for _, row in df_prod[df_prod['kind'].isin(['CA', 'CC'])].iterrows()
}

# Conversion des siren en strings dans les deux tables
df['codeEpci'] = df['codeEpci'].astype(str)
siren_to_insee = {
   str(row['siren']): row['localities_geo_code'] 
   for _, row in df_prod[df_prod['kind'].isin(['CA', 'CC'])].iterrows()
}

differences = []
for _, row in df.iterrows():
   code_insee = row['code'].replace("2A", "20").replace("2B", "20")
   code_epci_ref = str(row['codeEpci'])
   
   found = False
   for siren, insee_codes in siren_to_insee.items():
       if code_insee in insee_codes:
           if siren != code_epci_ref:
               differences.append({
                   'code_insee': code_insee,
                   'epci_ref': code_epci_ref,
                   'epci_prod': siren
               })
           found = True
           break
   
   if not found:
       differences.append({
           'code_insee': code_insee,
           'epci_ref': code_epci_ref,
           'epci_prod': None
       })

df_differences = pd.DataFrame(differences)
errors_by_epci = df_differences.groupby('epci_ref').size().reset_index(name='count')
print(f"Total erreurs: {len(df_differences)}")

Total erreurs: 2135


In [51]:
df_differences.query("epci_ref != '0'").to_csv('differences_to_upload.csv', index=False)

In [ ]:
errors_by_epci.to_csv("epci.csv", index=False)
!open .

In [ ]:
df_differences.to_csv("")

,code_insee,epci_ref,epci_prod
0,06006,200030195,Non trouvé
1,06009,200030195,Non trouvé
2,06011,200030195,Non trouvé
3,06013,200030195,Non trouvé
4,06020,200030195,Non trouvé
...,...,...,...
2130,98830,0,Non trouvé
2131,98831,0,Non trouvé
2132,98832,0,Non trouvé
2133,98833,0,Non trouvé


In [82]:
SIREN = "200067106"

In [85]:
df.query("epci_code == '200067106'")

,code,nom,siren,codeEpci,codeDepartement,codeRegion,codesPostaux,epci,departement,region,epci_code,epci_nom,departement_code,departement_nom,region_code,region_nom
24856,64008,Ahaxe-Alciette-Bascassan,216400085,200067106,64,75,[64220],"{'code': '200067106', 'nom': 'CA du Pays Basque'}","{'code': '64', 'nom': 'Pyrénées-Atlantiques'}","{'code': '75', 'nom': 'Nouvelle-Aquitaine'}",200067106,CA du Pays Basque,64,Pyrénées-Atlantiques,75,Nouvelle-Aquitaine
24857,64009,Ahetze,216400093,200067106,64,75,[64210],"{'code': '200067106', 'nom': 'CA du Pays Basque'}","{'code': '64', 'nom': 'Pyrénées-Atlantiques'}","{'code': '75', 'nom': 'Nouvelle-Aquitaine'}",200067106,CA du Pays Basque,64,Pyrénées-Atlantiques,75,Nouvelle-Aquitaine
24858,64010,Aïcirits-Camou-Suhast,216400101,200067106,64,75,[64120],"{'code': '200067106', 'nom': 'CA du Pays Basque'}","{'code': '64', 'nom': 'Pyrénées-Atlantiques'}","{'code': '75', 'nom': 'Nouvelle-Aquitaine'}",200067106,CA du Pays Basque,64,Pyrénées-Atlantiques,75,Nouvelle-Aquitaine
24859,64011,Aincille,216400119,200067106,64,75,[64220],"{'code': '200067106', 'nom': 'CA du Pays Basque'}","{'code': '64', 'nom': 'Pyrénées-Atlantiques'}","{'code': '75', 'nom': 'Nouvelle-Aquitaine'}",200067106,CA du Pays Basque,64,Pyrénées-Atlantiques,75,Nouvelle-Aquitaine
24860,64012,Ainharp,216400127,200067106,64,75,[64130],"{'code': '200067106', 'nom': 'CA du Pays Basque'}","{'code': '64', 'nom': 'Pyrénées-Atlantiques'}","{'code': '75', 'nom': 'Nouvelle-Aquitaine'}",200067106,CA du Pays Basque,64,Pyrénées-Atlantiques,75,Nouvelle-Aquitaine
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25379,64545,Urrugne,216405456,200067106,64,75,"[64122, 64700]","{'code': '200067106', 'nom': 'CA du Pays Basque'}","{'code': '64', 'nom': 'Pyrénées-Atlantiques'}","{'code': '75', 'nom': 'Nouvelle-Aquitaine'}",200067106,CA du Pays Basque,64,Pyrénées-Atlantiques,75,Nouvelle-Aquitaine
25380,64546,Urt,216405464,200067106,64,75,[64240],"{'code': '200067106', 'nom': 'CA du Pays Basque'}","{'code': '64', 'nom': 'Pyrénées-Atlantiques'}","{'code': '75', 'nom': 'Nouvelle-Aquitaine'}",200067106,CA du Pays Basque,64,Pyrénées-Atlantiques,75,Nouvelle-Aquitaine
25381,64547,Ustaritz,216405472,200067106,64,75,[64480],"{'code': '200067106', 'nom': 'CA du Pays Basque'}","{'code': '64', 'nom': 'Pyrénées-Atlantiques'}","{'code': '75', 'nom': 'Nouvelle-Aquitaine'}",200067106,CA du Pays Basque,64,Pyrénées-Atlantiques,75,Nouvelle-Aquitaine
25391,64558,Villefranque,216405589,200067106,64,75,[64990],"{'code': '200067106', 'nom': 'CA du Pays Basque'}","{'code': '64', 'nom': 'Pyrénées-Atlantiques'}","{'code': '75', 'nom': 'Nouvelle-Aquitaine'}",200067106,CA du Pays Basque,64,Pyrénées-Atlantiques,75,Nouvelle-Aquitaine


In [84]:
df_prod.query(f"siren == {SIREN}")["localities_geo_code"].iloc[0]

{'64008',
 '64009',
 '64010',
 '64011',
 '64012',
 '64013',
 '64014',
 '64015',
 '64016',
 '64017',
 '64018',
 '64019',
 '64024',
 '64026',
 '64031',
 '64034',
 '64035',
 '64036',
 '64038',
 '64045',
 '64046',
 '64047',
 '64049',
 '64050',
 '64051',
 '64065',
 '64066',
 '64081',
 '64086',
 '64092',
 '64093',
 '64094',
 '64100',
 '64102',
 '64105',
 '64106',
 '64107',
 '64113',
 '64115',
 '64120',
 '64122',
 '64123',
 '64124',
 '64125',
 '64130',
 '64134',
 '64140',
 '64147',
 '64150',
 '64154',
 '64155',
 '64160',
 '64161',
 '64162',
 '64166',
 '64187',
 '64188',
 '64189',
 '64202',
 '64213',
 '64214',
 '64218',
 '64221',
 '64222',
 '64228',
 '64229',
 '64231',
 '64235',
 '64247',
 '64249',
 '64250',
 '64255',
 '64256',
 '64258',
 '64259',
 '64260',
 '64264',
 '64265',
 '64267',
 '64268',
 '64271',
 '64272',
 '64273',
 '64274',
 '64275',
 '64277',
 '64279',
 '64282',
 '64283',
 '64285',
 '64289',
 '64294',
 '64297',
 '64298',
 '64303',
 '64304',
 '64313',
 '64314',
 '64316',
 '64317',


In [2]:
import pandas as pd
errors_by_epci = pd.read_csv("epci.csv")

In [3]:
errors_by_epci

,epci_ref,count
0,200015162.0,22
1,200023414.0,71
2,200027183.0,36
3,200030195.0,51
4,200033579.0,46
...,...,...
62,247100290.0,34
63,247200132.0,20
64,248300543.0,12
65,248719312.0,20


In [ ]:
df_differences

In [40]:
df_difference_old  = pd.read_csv("differences_epci.csv")

In [44]:
(df_difference_old["epci_ref"] == df_difference_old["epci_prod"]).sum()

29806

In [45]:
df_difference_old.shape

(29851, 3)